# Описание проекта
Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

        Шаги для выбора локации обычно такие:
- В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
- Строят модель для предсказания объёма запасов в новых скважинах;
- Выбирают скважины с самыми высокими оценками значений;
- Определяют регион с максимальной суммарной прибылью отобранных скважин.

Вам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Постройте модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой Bootstrap.

# Инструкция по выполнению проекта
1. Загрузите и подготовьте данные. Поясните порядок действий.
2. Обучите и проверьте модель для каждого региона:
- 2.1. Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
- 2.2. Обучите модель и сделайте предсказания на валидационной выборке.
- 2.3. Сохраните предсказания и правильные ответы на валидационной выборке.
- 2.4. Напечатайте на экране средний запас предсказанного сырья и RMSE модели.
- 2.5. Проанализируйте результаты.
3. Подготовьтесь к расчёту прибыли:
- 3.1. Все ключевые значения для расчётов сохраните в отдельных переменных.
- 3.2. Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.
- 3.3. Напишите выводы по этапу подготовки расчёта прибыли.
4. Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
- 4.1. Выберите скважины с максимальными значениями предсказаний.
- 4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
- 4.3. Рассчитайте прибыль для полученного объёма сырья.
5. Посчитайте риски и прибыль для каждого региона:
- 5.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
- 5.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
- 5.3. Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

# Описание данных
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

# Условия задачи:
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
- Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

# 1. Загрузка и подготовка данных

In [1]:
# Испортируем все нужные библиотки для дальнейшего анализа
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.utils import shuffle 
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [2]:
# Откроем таблицу двумя путями
server_oil_0 = '/datasets/geo_data_0.csv'
local_oil_0 = '/Users/mmaximmaximovgmail.com/Desktop/DS/project_8/geo_data_0.csv'

try:
    oil_0 = pd.read_csv(server_oil_0)
except:
    oil_0 = pd.read_csv(local_oil_0)
    
server_oil_1 = '/datasets/geo_data_1.csv'
local_oil_1 = '/Users/mmaximmaximovgmail.com/Desktop/DS/project_8/geo_data_1.csv'

try:
    oil_1 = pd.read_csv(server_oil_1)
except:
    oil_1 = pd.read_csv(local_oil_1)

server_oil_2 = '/datasets/geo_data_2.csv'
local_oil_2 = '/Users/mmaximmaximovgmail.com/Desktop/DS/project_8/geo_data_2.csv'

try:
    oil_2 = pd.read_csv(server_oil_2)
except:
    oil_2 = pd.read_csv(local_oil_2)

In [3]:
# Выводим рандомные 3 строки и типы/кол-во пропусков трех таблицы
display(oil_0.sample(3))
display(oil_0.info())
display(oil_1.sample(3))
display(oil_1.info())
display(oil_2.sample(3))
display(oil_2.info())

,id,f0,f1,f2,product
53751,4hfp3,0.468881,-0.332820,0.943622,173.709369
95024,qzFN6,0.367129,1.043043,4.052270,74.971991
44717,o8W0Z,0.215729,-0.091112,0.142734,36.968139


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
5894,rosKk,-6.952827,-9.017009,2.997788,84.038886
11138,Sbt4M,7.744215,-8.172707,5.000058,134.766305
46074,N5uxp,2.971350,-12.042381,1.998653,57.085625


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
34418,uAPQe,2.454177,-3.020657,4.745727,166.217323
28940,ul26s,1.735628,0.271190,3.243407,17.776241
40633,y8V7b,-0.129294,1.989403,4.574698,81.957168


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

In [4]:
# Видим, что с даными все в порядке, но для более хорошей работы, я удалю ненужную колонку id с каждой таблицы
# inplace=True помогает сразу внести изменения в ту же самую перменную
oil_0.drop(['id'], axis=1, inplace=True)
oil_1.drop(['id'], axis=1, inplace=True)
oil_2.drop(['id'], axis=1, inplace=True)
# Проверяем, удалилась ли колонка
display(oil_0.shape)
display(oil_1.shape)
display(oil_2.shape)

(100000, 4)

(100000, 4)

(100000, 4)

### Мини вывод по 1 шагу
- Все данные проверенны, нет пропусков и типы колонок верны
- Удалили не нужную колонку

# 2. Обучение и проверка модели

In [5]:
# Разбиваем данные на выборку обущающей и валидационную, в соотношении 75:25

# Сначала для oil_0

feature_0 = oil_0.drop('product', axis=1)
target_0 = oil_0['product']

feature_train_0, feature_valid_0, target_train_0, target_valid_0 = train_test_split(
    feature_0, target_0, random_state=12345, test_size=0.25
)
display(feature_train_0.shape, target_train_0.shape)
display(feature_valid_0.shape, target_valid_0.shape)

(75000, 3)

(75000,)

(25000, 3)

(25000,)

In [6]:
# Создаем модель где найдем средний запас через mean и RMSE модели для oil_0
model = LinearRegression()
model.fit(feature_train_0, target_train_0)
prediction_0 = model.predict(feature_valid_0)
#prediction_0 = pd.Series(prediction_0, index=target_0.index)
result = mean_squared_error(target_valid_0, prediction_0) ** 0.5
print('RMSE модельной регрессии на валидационной выборке:', result)
print('Mean средний запас предсказанного сырья:', prediction_0.mean())

RMSE модельной регрессии на валидационной выборке: 37.5794217150813
Mean средний запас предсказанного сырья: 92.59256778438035


In [7]:
# Разбиваем данные на выборку обущающей и валидационную, в соотношении 75:25

# Для oil_1

feature_1 = oil_1.drop('product', axis=1)
target_1 = oil_1['product']

feature_train_1, feature_valid_1, target_train_1, target_valid_1 = train_test_split(
    feature_1, target_1, random_state=12345, test_size=0.25
)
display(feature_train_1.shape, target_train_1.shape)
display(feature_valid_1.shape, target_valid_1.shape)

(75000, 3)

(75000,)

(25000, 3)

(25000,)

In [8]:
# Создаем модель где найдем средний запас через mean и RMSE модели для oil_1
model = LinearRegression()
model.fit(feature_train_1, target_train_1)
prediction_1 = model.predict(feature_valid_1)
#prediction_1 = pd.Series(prediction_1)
result = mean_squared_error(target_valid_1, prediction_1) ** 0.5
print('RMSE модельной регрессии на валидационной выборке:', result)
print('Mean средний запас предсказанного сырья:', prediction_1.mean())

RMSE модельной регрессии на валидационной выборке: 0.893099286775617
Mean средний запас предсказанного сырья: 68.728546895446


In [9]:
# Разбиваем данные на выборку обущающей и валидационную, в соотношении 75:25

# Для oil_2

feature_2 = oil_2.drop('product', axis=1)
target_2 = oil_2['product']

feature_train_2, feature_valid_2, target_train_2, target_valid_2 = train_test_split(
    feature_2, target_2, random_state=12345, test_size=0.25
)
display(feature_train_2.shape, target_train_2.shape)
display(feature_valid_2.shape, target_valid_2.shape)

(75000, 3)

(75000,)

(25000, 3)

(25000,)

In [10]:
# Создаем модель где найдем средний запас через mean и RMSE модели для oil_2
model = LinearRegression()
model.fit(feature_train_2, target_train_2)
prediction_2 = model.predict(feature_valid_2)
#prediction_2 = pd.Series(prediction_2)
result = mean_squared_error(target_valid_2, prediction_2) ** 0.5
print('RMSE модельной регрессии на валидационной выборке:', result)
print('Mean средний запас предсказанного сырья:', prediction_2.mean())

RMSE модельной регрессии на валидационной выборке: 40.02970873393434
Mean средний запас предсказанного сырья: 94.96504596800489


### Мини вывод по 2 шагу
- Худший показатель по RMSE: 1 и 3 регион, тобишь oil_0 и oil_2
- Лучший показатель Mean: так же 3 регион

# 3. Подготовка к расчёту прибыли

In [11]:
# Исследуем 500 точек
LOC = 500
# 200 лучших точек
BEST_LOC = 200
# Бюджет 10 млрд, возводим 10 в 10 степени
MONEY = 10**10
# Каждая единица продукта состовляет 450 тыс.
BARREL = 450 * 10**3

In [12]:
# Расчитаем средний сырья в каждом регионе
print('Cредний запас в 1 регионе:', oil_0['product'].mean())
print('Cредний запас в 2 регионе:', oil_1['product'].mean())
print('Cредний запас в 3 регионе:', oil_2['product'].mean())

Cредний запас в 1 регионе: 92.50000000000001
Cредний запас в 2 регионе: 68.82500000000002
Cредний запас в 3 регионе: 95.00000000000004


In [13]:
# Стоимость разработки одного месторождения
one_point = MONEY/BEST_LOC
print(f'Стоимость разработки одного из лучших месторождений: {one_point:.0f}')
# Точка безубытоности 
no_loss = one_point / BARREL
print(f'Точка безубыточности: {no_loss:.0f}')
# Средний обьем нефти для региона в безубыточности 
no_loss_loc = MONEY/BARREL
print('Средний обьем нефти для региона в безубыточности:', round(no_loss_loc))

Стоимость разработки одного из лучших месторождений: 50000000
Точка безубыточности: 111
Средний обьем нефти для региона в безубыточности: 22222


# 4. Расчёт прибыли и рисков 

In [14]:
# Создаем бутстреп для 1 региона

def revenue(target_0, probabilities, count):
    probabilities = pd.Series(probabilities, index=target_0.index)
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target_0[probs_sorted.index][:count]
    return selected.sum() * BARREL - MONEY

state = np.random.RandomState(12345)  
    
values_0 = []
risk_0 = 0
for i in range(1000):
    target_subsample_0 = target_valid_0.reset_index(drop=True).sample(500, replace=True, random_state=state)
    pred_subsample_0 = prediction_0[target_subsample_0.index]
    res_0 = revenue(target_subsample_0, pred_subsample_0, 200)
    values_0.append(res_0)
    if res_0 < 0:
        risk_0 += 1

values_0 = pd.Series(values_0)
lower_0 = round(values_0.quantile(0.025))
upper_0 = round(values_0.quantile(0.975))
mean_0 = round(values_0.mean())
risk_0 = risk_0 / 1000

print('Вся информация по первой локции')
print()
print("Средняя выручка:", mean_0)
print("5%-квантиль:", lower_0)
print("95%-квантиль:", upper_0)
print(f'Процент риска: {risk_0:.1%}')

Вся информация по первой локции

Средняя выручка: 425938527
5%-квантиль: -102090095
95%-квантиль: 947976353
Процент риска: 6.0%


In [15]:
# Создаем бутстреп для 2 региона

def revenue(target_1, probabilities, count):
    probabilities = pd.Series(probabilities, index=target_1.index)
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target_1[probs_sorted.index][:count]
    return selected.sum() * BARREL - MONEY

state = np.random.RandomState(12345)  
    
values_1 = []
risk_1 = 0
for i in range(1000):
    target_subsample_1 = target_valid_1.reset_index(drop=True).sample(500, replace=True, random_state=state)
    pred_subsample_1 = prediction_1[target_subsample_1.index]
    res_1 = revenue(target_subsample_1, pred_subsample_1, 200)
    values_1.append(res_1)
    if res_1 < 0:
        risk_1 += 1

values_1 = pd.Series(values_1)
lower_1 = round(values_1.quantile(0.025))
upper_1 = round(values_1.quantile(0.975))
mean_1 = round(values_1.mean())
risk_1 = risk_0 / 1000

print('Вся информация по второй локции')
print()
print("Средняя выручка:", mean_1)
print("5%-квантиль:", lower_1)
print("95%-квантиль:", upper_1)
print(f'Процент риска: {risk_1:.1%}')

Вся информация по второй локции

Средняя выручка: 515222773
5%-квантиль: 68873225
95%-квантиль: 931547591
Процент риска: 0.0%


In [16]:
# Создаем бутстреп для 3 региона

def revenue(target_2, probabilities, count):
    probabilities = pd.Series(probabilities, index=target_2.index)
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target_2[probs_sorted.index][:count]
    return selected.sum() * BARREL - MONEY

state = np.random.RandomState(12345)  
    
values_2 = []
risk_2 = 0
for i in range(1000):
    target_subsample_2 = target_valid_2.reset_index(drop=True).sample(500, replace=True, random_state=state)
    pred_subsample_2 = prediction_2[target_subsample_2.index]
    res_2 = revenue(target_subsample_2, pred_subsample_2, 200)
    values_2.append(res_2)
    if res_2 < 0:
        risk_2 += 1

values_2 = pd.Series(values_2)
lower_2 = round(values_2.quantile(0.025))
upper_2 = round(values_2.quantile(0.975))
mean_2 = round(values_2.mean())
risk_2 = risk_2 / 1000

print('Вся информация по третьей локции')
print()
print("Средняя выручка:", mean_2)
print("5%-квантиль:", lower_2)
print("95%-квантиль:", upper_2)
print(f'Процент риска: {risk_2:.1%}')

Вся информация по третьей локции

Средняя выручка: 435008363
5%-квантиль: -128880547
95%-квантиль: 969706954
Процент риска: 6.4%


# 5. Вывод

В данной работе главное задачей было обучить модель для каждого региона, и что я получил:
- Во первых это конечно загрузили все нужные мне библиотеки, присвоил для каждого региона свои перменные, сделал первичный анализ и удалил не нужные строки в столбцах
- Далее я для каждого региона разбил данные на выборки: обучающую и валидационную, в соотношение 75:25
- После выборки я посчитал среднее значение (mean) и RMSE, так же для каждого региона
- В третьем шаге я присвоил новые переменные и расчитал:
    1. Точку безубыточности
    2. Стоимость разработки одной из лучших точек
    3. Средний обьем нефти для региона в безубыточности 
- И в последнем шагу было расчитанна: средняя выручка, 5% и 95% квантиль, процент риска
- По итогу, с наименьшим процентом риска и наибольшей средней выручкой, выходит 2 регион или geo_data_1, она показала наиболее лучшие значения чем 1 и 3 регион